In [2]:
!pip install gensim nltk rouge-score

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 15.1 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=ae1096528565d9b89eaf1a6fffd21b57d79def7fded1db2caab6a76de5cb92b2
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Suc

In [4]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/MyDrive/M Tech/Sem3")

Mounted at /content/drive


In [1]:
import json
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from sklearn.metrics import f1_score

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
w2v_path = "Embeddings/cc.hi.300.vec"
w2v_model = KeyedVectors.load_word2vec_format(w2v_path)

In [7]:
def sentence_vector(sentence, model):
    words = word_tokenize(sentence.lower())
    word_vecs = [model[word] for word in words if word in model]
    return np.mean(word_vecs, axis=0) if word_vecs else np.zeros(model.vector_size)

In [8]:
def load_dataset(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        raw = json.load(f)
    samples = []
    for domain in raw["domains"]:
        for item in domain["contexts"]:
            context = item["context"]
            for qa in item["qas"]:
                samples.append({
                    "id": qa["id"],
                    "context": context,
                    "question": qa["question"],
                    "answer": qa["answer"]
                })
    return pd.DataFrame(samples)


In [10]:
def predict_answer(context, question):
    sentences = sent_tokenize(context)
    sentence_vecs = [sentence_vector(sent, w2v_model) for sent in sentences]
    q_vec = sentence_vector(question, w2v_model)
    similarities = cosine_similarity([q_vec], sentence_vecs)[0]

    return sentences[np.argmax(similarities)]

def calculate_metrics(samples):

    smoothie = SmoothingFunction().method4
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

    f1s, bleus, rougels = [], [], []

    for _, row in samples.iterrows():
        pred = row['predicted']
        true = row['answer']

        # F1
        pred_tokens = word_tokenize(pred)
        true_tokens = word_tokenize(true)
        common = set(pred_tokens) & set(true_tokens)
        if len(pred_tokens) == 0 or len(true_tokens) == 0:
            f1 = 0
        else:
            precision = len(common) / len(pred_tokens)
            recall = len(common) / len(true_tokens)
            f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        f1s.append(f1)

        # BLEU
        bleus.append(sentence_bleu([true_tokens], pred_tokens, smoothing_function=smoothie))

        # ROUGE-L
        rouge = scorer.score(true, pred)
        rougels.append(rouge['rougeL'].fmeasure)

    return {
        "F1": np.mean(f1s),
        "BLEU": np.mean(bleus),
        "ROUGE-L": np.mean(rougels)
    }


In [13]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [18]:
df = load_dataset("Dataset/train.json")
df["predicted"] = df.apply(lambda row: predict_answer(row["context"], row["question"]), axis=1)

df.to_csv("Embeddings/Word2Vec_results_train.csv", index=False, encoding='utf-8')
df[["id", "question", "answer", "predicted"]].to_json("results.json", force_ascii=False, indent=2)

In [19]:
# Print Evaluation
metrics = calculate_metrics(df)
print("Evaluation Metrics:")

for k, v in metrics.items():
    print(f"{k}: {v:.4f}")

Evaluation Metrics:
F1: 0.2672
BLEU: 0.1141
ROUGE-L: 0.2272


In [20]:
df = load_dataset("Dataset/validation.json")
df["predicted"] = df.apply(lambda row: predict_answer(row["context"], row["question"]), axis=1)

df.to_csv("Embeddings/Word2Vec_results_val.csv", index=False, encoding='utf-8')
df[["id", "question", "answer", "predicted"]].to_json("results.json", force_ascii=False, indent=2)

In [ ]:
# Print Evaluation
metrics = calculate_metrics(df)
print("Evaluation Metrics:")

for k, v in metrics.items():
    print(f"{k}: {v:.4f}")

Evaluation Metrics:
F1: 0.2668
BLEU: 0.1144
ROUGE-L: 0.2190


In [ ]:
df = load_dataset("Dataset/test-A-gold.json")
df["predicted"] = df.apply(lambda row: predict_answer(row["context"], row["question"]), axis=1)

df.to_csv("Embeddings/Word2Vec_results_test.csv", index=False, encoding='utf-8')
df[["id", "question", "answer", "predicted"]].to_json("results.json", force_ascii=False, indent=2)

In [ ]:
# Print Evaluation
metrics = calculate_metrics(df)
print("Evaluation Metrics:")

for k, v in metrics.items():
    print(f"{k}: {v:.4f}")

Evaluation Metrics:
F1: 0.2668
BLEU: 0.1144
ROUGE-L: 0.2190
